<a href="https://colab.research.google.com/github/Amarmurun0212/Diver/blob/main/Credit_information_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

def Encoder(df):
    columnsToEncode = list(df.select_dtypes(include=['category','object']))
    le = LabelEncoder()
    for feature in columnsToEncode:
        try:
            df[feature] = le.fit_transform(df[feature])
        except:
            print('Error encoding '+feature)
    return df


[Problem 1] Confirmation of the content of the competition

1. what to learn and what to predict
Learn about home home credit data and predict default risk.
2. What kind of file to create and submit to Kaggle
For each SK_ID_CURR in the test set, must predict a probability for the TARGET variable.
3. What kind of index value will the submitted work be evaluated by?
Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.

[Problem 2] Learning and verification

In [5]:
application_train = pd.read_csv('application_train.csv')
application_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60157,169755,0,Cash loans,F,N,Y,0,157500.0,970380.0,28503.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60158,169756,0,Cash loans,M,N,Y,0,135000.0,157500.0,17811.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
60159,169757,0,Cash loans,M,N,Y,0,202500.0,1125000.0,32895.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
60160,169758,0,Revolving loans,F,N,N,0,180000.0,337500.0,16875.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
application_test = pd.read_csv('application_test.csv')
application_test

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [7]:
application_train.dropna(axis = 1, how='any', inplace=True)
y_train = application_train['TARGET']
application_train = application_train.drop("TARGET", axis=1)
print("after drop features with NA or null values from application_train:{}".format(len(application_train)))
print("features used in train:{}".format(application_train.columns))
print('Feature data types:{}'.format(application_train.dtypes))

application_test = application_test[application_train.columns]
print(application_train.head(10))

after drop features with NA or null values from application_train:60162
features used in train:Index(['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'NAME_INCOME_TYPE'],
      dtype='object')
Feature data types:SK_ID_CURR              int64
NAME_CONTRACT_TYPE     object
CODE_GENDER            object
FLAG_OWN_CAR           object
FLAG_OWN_REALTY        object
CNT_CHILDREN            int64
AMT_INCOME_TOTAL      float64
AMT_CREDIT            float64
NAME_INCOME_TYPE       object
dtype: object
   SK_ID_CURR NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  \
0      100002         Cash loans           M            N               Y   
1      100003         Cash loans           F            N               N   
2      100004    Revolving loans           M            Y               Y   
3      100006         Cash loans           F            N               Y   
4      100007    

In [8]:
application_train = Encoder(application_train)
application_test = Encoder(application_test)

application_train = pd.get_dummies(application_train)
application_test = pd.get_dummies(application_test)
# training 
scaler = StandardScaler()
scaler.fit(application_train)
X_train = scaler.transform(application_train)
X_test = scaler.transform(application_test)

x_tr, x_val, y_tr, y_val = train_test_split(X_train, y_train, random_state=42)

lr = LogisticRegression(C=1)
lr.fit(x_tr, y_tr)
print(y_tr)

<ipython-input-2-2fef1a72299e>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = le.fit_transform(df[feature])
<ipython-input-2-2fef1a72299e>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = le.fit_transform(df[feature])
<ipython-input-2-2fef1a72299e>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

30371    0
18349    0
1370     0
47774    0
56396    0
        ..
54343    0
38158    0
860      0
15795    1
56422    0
Name: TARGET, Length: 45121, dtype: int64


[Problem 3] Estimation for test data

In [9]:
y_test_pred = lr.predict_proba(x_val)[:, 1]
print("roc auc score", roc_auc_score(y_val, y_test_pred))

y_test_pred = lr.predict_proba(X_test)[:, 1]
data = application_test[["SK_ID_CURR"]]
data['TARGET'] = y_test_pred.tolist()
data.to_csv('submission.csv', index=False)

roc auc score 0.5902445481579923


<ipython-input-9-5366070d1b93>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TARGET'] = y_test_pred.tolist()


[Problem 4] Feature quantity engineering

In [10]:
application_train = pd.read_csv('application_train.csv')
application_test = pd.read_csv('application_test.csv')
print("Number of samples in application_train:{}".format(len(application_train)))
print("Number of features in application_train:{}".format(len(application_train.columns)))
print("Number of samples in application_test:{}".format(len(application_test)))
print("Number of features in application_test:{}".format(len(application_test.columns)))


correlations = application_train.corr()["TARGET"].sort_values()

print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

'''
From above result, We can see that the most correlated variables are DAYS_BIRTH, EXT_SOURCE_3, EXT_SOURCE_2 and EXT_SOURCE_1.     
'''

ext_data = application_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
print(ext_data_corrs)


y_ext = ext_data['TARGET']
X_train = ext_data.drop(columns=['TARGET'])
X_test = application_test[X_train.columns]

#print(X_train.columns)

imputer = SimpleImputer(strategy="mean")
X_train = imputer.fit_transform(X_train)
X_test = imputer.fit_transform(X_test)

scaler = StandardScaler()
scaler.fit(X_train)
train = scaler.transform(X_train)
test = scaler.transform(X_test)


x_tr, x_val, y_tr, y_val = train_test_split(train, y_ext, random_state=42)

log_reg = LogisticRegression(C=1)
log_reg.fit(x_tr, y_tr)

log_reg_pred = log_reg.predict_proba(x_val)[:, 1]
print("roc auc score", roc_auc_score(y_val, log_reg_pred))

y_test_pred = log_reg.predict_proba(X_test)[:, 1]

data = application_test[["SK_ID_CURR"]]
data['TARGET'] = y_test_pred.tolist()
data.to_csv('submission.csv', index=False)

Number of samples in application_train:98949
Number of features in application_train:122
Number of samples in application_test:48744
Number of features in application_test:121


<ipython-input-10-f34d2e134a07>:9: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlations = application_train.corr()["TARGET"].sort_values()


Most Positive Correlations:
 LIVE_CITY_NOT_WORK_CITY        0.033471
DEF_30_CNT_SOCIAL_CIRCLE       0.035338
DEF_60_CNT_SOCIAL_CIRCLE       0.036324
REG_CITY_NOT_LIVE_CITY         0.036915
DAYS_REGISTRATION              0.039339
OWN_CAR_AGE                    0.044036
FLAG_DOCUMENT_3                0.044593
FLAG_EMP_PHONE                 0.045064
REG_CITY_NOT_WORK_CITY         0.048757
DAYS_ID_PUBLISH                0.052195
DAYS_LAST_PHONE_CHANGE         0.052890
REGION_RATING_CLIENT           0.061927
REGION_RATING_CLIENT_W_CITY    0.063198
DAYS_BIRTH                     0.077342
TARGET                         1.000000
Name: TARGET, dtype: float64

Most Negative Correlations:
 EXT_SOURCE_3                 -0.179060
EXT_SOURCE_2                 -0.161708
EXT_SOURCE_1                 -0.153969
FLOORSMAX_AVG                -0.044718
FLOORSMAX_MEDI               -0.044253
DAYS_EMPLOYED                -0.044087
FLOORSMAX_MODE               -0.043630
AMT_GOODS_PRICE              -0.040043


<ipython-input-10-f34d2e134a07>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TARGET'] = y_test_pred.tolist()
